In [11]:
import pandas as pd
import numpy as np
import random
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression, LinearRegression
import numpy as np
import math
from sklearn import metrics
from sklearn import preprocessing
from geopy.distance import vincenty
import seaborn 

In [12]:
#--------------------------------------------------------------------------------------------------------
#Loading the dataframes that we will need:
df = pd.read_csv('datasets/completdf')
df1 = pd.read_csv('datasets/mergedf')
dfmtr = pd.read_csv("datasets/MTR_List")

#Create dicts for lat and long
dictlat = dict(zip(df.ref, df.lat))
dictlng = dict(zip(df.ref, df.lng))

df1["lat"] = df1.ref.map(dictlat)
df1["lng"] = df1.ref.map(dictlng)

del df1["Unnamed: 0"]
del df1["sell"]
del df1["mortgage"]

#Replacing empty values by 0:
df1 = df1[df1.rent <= 1000000]
df1 = df1.fillna(0)
df1 = df1.replace("-", 0)
df1 = df1.replace("--", 0)
df1.gross_ar = df1.gross_ar.replace(1, 0)
df1.salar_ar = df1.salar_ar.replace(1, 0)

#Change the type of the data and solving some issues:
df1["gross_ar"] = df1["gross_ar"].astype(float)
df1["salar_ar"] = df1["salar_ar"].astype(float)
df1["bath"] = df1["bath"].astype(float)
df1["age"] = df1["age"].astype(float)
df1 = df1[df1.ref != 19954239]
df1 = df1[df1.ref != 21404689]
df1.bedroom = df1.bedroom.replace({127 : 0, 34 : 0, 24 : 0})

#Dict with reference, gross area and saleable -> Calculate the ratio gross/saleable area
dictarea = dict(zip(df1.gross_ar, df1.salar_ar))
d_ref_gross_sal= dict([(i,[a,b ]) for i, a,b in zip(df1.ref, df1.gross_ar,df1.salar_ar)])


gros_div_sal = 0

for i, j in dictarea.items():
    if j != 0 and i != 0:
        a = (j/i)
        gros_div_sal += a
        
ratio_gross_sal = gros_div_sal/len(dictarea)


In [ ]:
#!!!!!!!!!!!!!!!!!A!!!!!!!!!!!!!!
#--------------------------------------------------------------------------------------------------------
#Fill empty values in gross/saleable columns:
gross_ar = []
salar_ar = []
ref = []
for i, j in d_ref_gross_sal.itervalues():
    if i != 0 or j != 0:
        if i == 0:
            gross_ar.append((j)/ratio_gross_sal)
            
        else:
            gross_ar.append((i)) 
            
        if j == 0:
            salar_ar.append((i)*ratio_gross_sal)
            
        else:
            salar_ar.append((j))
    elif i != 0 and j != 0:
        if j == 65535:
            gross_ar.append(i)
            salar_ar.append(i * ratio_gross_sal)
        elif i == 1:
            gross_ar.append(j/ratio_gross_sal)
            salar_ar.append(j)
        elif j == 1:
            gross_ar.append(i)
            salar_ar.append(i * ratio_gross_sal)
        else:
            gross_ar.append(i)
            salar_ar.append(j)
            
            
    else:     
        gross_ar.append(gross_mean)
        salar_ar.append(salar_ar_mean)
    
for t in d_ref_gross_sal.iteritems():
    ref.append(t[0])

In [ ]:
#!!!!!!!!!!!!!B!!!!!!!!!!!!!!!
#--------------------------------------------------------------------------------------------------------
#!!!!************ONCE YOU EXECUTE THE CELL ABOVE, EXECUTE THIS CELL AND RETURN BACK TO /\ AND EXECUTE IT. AFTER THAT, NO NEED TO
#RUN AGAIN THIS CELL, GO TO THE NEXT ONE BELOW \/
#Calculate the ratio gross/saleable area to use it in the the cell above /\

t = 0
p = 0
for i in gross_ar:
    if i != 0:
        x = i
        t += x
        p += 1
d = 0
c = 0
for i in salar_ar:
    if i != 0:
        o = i
        d += o
        c += 1

gross_mean = t/p
salar_ar_mean = d/c

In [ ]:
#--------------------------------------------------------------------------------------------------------
#Cleaning and filling empty values on our features:

#Dicts with saleable/gross area:
dict_salar = dict(zip(ref, salar_ar))
dict_gross = dict(zip(ref, gross_ar))

df1["gross"] = df1.ref.map(dict_gross)
df1["salar"] = df1.ref.map(dict_salar)

del df1["gross_ar"]
del df1["salar_ar"]

df1["type"] = df1.proptype.map({"Unit / Flat / Apartment :" : "Flat", "Village house :" : "House", "House / Villa :" : "House", "Boat :" : "House", "Residential :" : "Flat"})

del df1["proptype"]


flat = {"High" : 470, "Mid" : 340, "Low": 188, "Very High" : 2}
d_ref_floor_type = dict([(i,[a,b ]) for i, a,b in zip(df1.ref, df1.floor,df1.type)])

#Filling empty values for floor level, it depends on the property type:
floorlvl = []
ref2 = []
for i, j in d_ref_floor_type.itervalues():
    if j == "Flat":
        if i == 0:
            ran = random.choice([x for x in flat for y in range(flat[x])])
            floorlvl.append(ran)
        elif pd.isnull(i):
            rand = random.choice([x for x in flat for y in range(flat[x])])
            floorlvl.append(rand)
        else:
            floorlvl.append(i)
    elif j == "House":
        if i == 0:
            floorlvl.append("G/F")
        else:
            floorlvl.append(i)

for t in d_ref_floor_type.iteritems():
    ref2.append(t[0])
    
dictfloor = dict(zip(ref2, floorlvl))
df1["floortype"] = df1.ref.map(dictfloor)

dfloor = df1[df1.type == "Flat"]
dfhouse = df1[df1.type == "House"]


#Filling the age column, counting the % for ages with a range(0, 100, 20) -- FLAT:
vent = 0
cuar = 0
sese = 0
oche = 0
cien = 0
cont = 0
for i in dfloor.age:
    if i != 0:
        if i < 20:
            x = i
            vent += x
            cont +=1
        elif i >= 20 and i < 40:
            y = i
            cuar += y
            cont +=1
        elif i >= 40 and i < 60:
            t = i
            sese += t
            cont +=1
        elif i >= 60 and int(i) < 80:
            p = i
            oche += p
            cont +=1
        elif i > 80:
            a = i
            cien += a
            cont +=1

total = vent + cuar +sese + oche + cien
            
print "20 range:", vent/float(total)
print "40 range:", cuar/float(total)
print "60 range:", sese/float(total)
print "80 range:", oche/float(total)
print "100 range:", cien/float(total)


#Filling the age column, counting the % for ages with a range(0, 100, 20) -- HOUSE:
vent = 0
cuar = 0
sese = 0
oche = 0
cien = 0
cont = 0
for i in dfhouse.age:
    if i != 0:
        if i < 20:
            x = i
            vent += x
            cont +=1
        elif i >= 20 and i < 40:
            y = i
            cuar += y
            cont +=1
        elif i >= 40 and i < 60:
            t = i
            sese += t
            cont +=1
        elif i >= 60 and i < 80:
            p = i
            oche += p
            cont +=1
        elif i > 80:
            a = i
            cien += a
            cont +=1
            
total = vent + cuar +sese + oche + cien

print "20 range:", vent/float(total)
print "40 range:", cuar/float(total)
print "60 range:", sese/float(total)
print "80 range:", oche/float(total)
print "100 range:", cien/float(total)

#Probability for the ages:
ageranflat = {10 : 163, 20 : 438, 60: 364, 80: 26, 100 :6}
ageranhouse = {10 : 14, 20 : 65, 60: 13, 80: 1, 100 :4}

#Filling empty values choosing a random number using the % above /\:
d_ref_age_type = dict([(i,[a,b ]) for i, a,b in zip(df1.ref, df1.age,df1.type)])

agelist = []
ref3 = []
for i, j in d_ref_age_type.itervalues():
    if j == "Flat":
        if i == 0:
            ran = random.choice([x for x in ageranflat for y in range(ageranflat[x])])
            agelist.append(ran)
        else:
            agelist.append(i)
    else:
        if j == "House":
            if i == 0:
                rand = random.choice([x for x in ageranhouse for y in range(ageranhouse[x])])
                agelist.append(rand)
            else:
                floorlvl.append(i)

for t in d_ref_age_type.iteritems():
    ref3.append(t[0]) 
    

dictages = dict(zip(ref3, agelist))
df1["age_building"] = df1.ref.map(dictages)
del df1["age"]
dflat = df1[df1["type"] == "Flat"]
dfcasa = df1[df1.type == "House"]

#Depending on the size of the property, count the number of bedrooms and baths, and fill the empty values using the ratios -- FLAT:
d_ref_bed_bath_gross= dict([(i,[a,b,c ]) for i, a,b,c in zip(dflat.ref, dflat.bedroom,dflat.bath, dflat.gross)])

bed500 = 0
bat500 = 0
tot500 = 0
bed1000 = 0
bat1000 = 0
tot1000 = 0
bed1500 = 0
bat1500 = 0
tot1500 = 0
bed2000 = 0
bat2000 = 0
tot2000 = 0
bed2500 = 0
bat2500 = 0
tot2500 = 0
bed3000 = 0
bat3000 = 0
tot3000 = 0
bed3500 = 0
bat3500 = 0
tot3500 = 0


for x, y, z in d_ref_bed_bath_gross.itervalues():
    if z < 500:
        if x != 0:
            if y != 0:
                bed500 += x
                bat500 += y
            tot500 += 1
    if z >= 500 and z < 1000:
        if x != 0:
            if y != 0:
                bed1000 += x
                bat1000 += y
            tot1000 += 1
    if z >= 1000 and z < 1500:
        if x != 0:
            if y != 0:
                bed1500 += x
                bat1500 += y
            tot1500 += 1
    if z >= 1500 and z < 2000:
        if x != 0:
            if y != 0:
                bed2000 += x
                bat2000 += y
            tot2000 += 1
    if z >= 2000 and z < 2500:
        if x != 0:
            if y != 0:
                bed2500 += x
                bat2500 += y
            tot2500 += 1
    if z >= 2500 and z < 3000:
        if x != 0:
            if y != 0:
                bed3000 += x
                bat3000 += y
            tot3000 += 1
    if z >= 3000:
        if x != 0:
            if y != 0:
                bed3500 += x
                bat3500 += y
            tot3500 += 1

ratio500 = bed500/bat500
ratio1000 = bed1000/bat1000
ratio1500 = bed1500/bat1500
ratio2000 = bed2000/bat2000
ratio2500 = bed2500/bat2500
ratio3000 = bed3000/bat3000
ratio3500 = bed3500/bat3500
bedsin500 = bed500/tot500
bedsin1000 = bed1000/tot1000
bedsin1500 = bed1500/tot1500
bedsin2000 = bed2000/tot2000
bedsin2500 = bed2500/tot2500
bedsin3000 = bed3000/tot3000
bedsin3500 = bed3500/tot3500

print "Ratio 500:", ratio500, "beds:", bedsin500
print "Ratio 1000:", ratio1000, "beds:", bedsin1000
print "Ratio 1500:", ratio1500, "beds:", bedsin1500
print "Ratio 2000:", ratio2000, "beds:", bedsin2000
print "Ratio 2500:", ratio2500, "beds:", bedsin2500
print "Ratio 3000:", ratio3000, "beds:", bedsin3000
print "Ratio 3500:", ratio3500, "beds:", bedsin3500


casbed = 0
casbat = 0
totbed = 0
for i in dfcasa.bedroom:
    if i != 0:
        casbed += i
    totbed +=1
for j in dfcasa.bath:
    if j != 0:
        casbat += int(j)
cas_bed_bath = casbed/casbat
print cas_bed_bath
total_beds = casbed/totbed
print total_beds

d_ref_bed_bat_gross_type= dict([(i,[a,b,c,d ]) for i, a,b,c,d in zip(df1.ref, df1.bedroom,df1.bath, df1.gross, df1.type)])


#Depending on the size of the property, count the number of bedrooms and baths, and fill the empty values using the ratios -- House:
habitaciones = []
bannos = []
ref4 = []
for x, y, z, w in d_ref_bed_bat_gross_type.itervalues():
    if w == "House":
        if x != 0 and y != 0:
            bannos.append(y)
            habitaciones.append(x)
        elif x == 0 and y != 0:
            bannos.append(y)
            habitaciones.append(round(float(cas_bed_bath*y)))
        elif x != 0 and y == 0:
            bannos.append(round(x/cas_bed_bath))
            habitaciones.append(x)
        else:
            bannos.append((round(total_beds)/cas_bed_bath))
            habitaciones.append(round(total_beds))
            
    else:
        if z < 500:
            if x != 0 and y != 0:
                bannos.append(y)
                habitaciones.append(x)
            elif x==0 and y != 0:
                bannos.append(y)
                habitaciones.append(round(float(y*ratio500)))
            elif x != 0 and y == 0:
                bannos.append(round(x/ratio500))
                habitaciones.append(x)
            else:
                bannos.append(round(bedsin500/ratio500))
                habitaciones.append(round(bedsin500))
        elif 500 >= z and z<1000:
            if x != 0 and y != 0:
                bannos.append(y)
                habitaciones.append(x)
            elif x==0 and y != 0:
                bannos.append(y)
                habitaciones.append(round(float(y*ratio1000)))
            elif x != 0 and y == 0:
                bannos.append(round(x/ratio1000))
                habitaciones.append(x)
            else:
                bannos.append(round(bedsin1000/ratio1000))
                habitaciones.append(round(bedsin1000))
        elif 1000 >= z and z <1500:
            if x != 0 and y != 0:
                bannos.append(y)
                habitaciones.append(x)
            elif x==0 and y != 0:
                bannos.append(y)
                habitaciones.append(round(float(y*ratio1500)))
            elif x != 0 and y == 0:
                bannos.append(round(x/ratio1500))
                habitaciones.append(x)
            else:
                bannos.append(round(bedsin1500/ratio1500))
                habitaciones.append(round(bedsin1500))
        elif 1500 >= z and z < 2000:
            if x != 0 and y != 0:
                bannos.append(y)
                habitaciones.append(x)
            elif x==0 and y != 0:
                bannos.append(y)
                habitaciones.append(round(float(y*ratio2000)))
            elif x != 0 and y == 0:
                bannos.append(round(x/ratio2000))
                habitaciones.append(x)
            else:
                bannos.append(round(bedsin2000/ratio2000))
                habitaciones.append(round(bedsin2000))
        elif 2000 >= z and z < 2500:
            if x != 0 and y != 0:
                bannos.append(y)
                habitaciones.append(x)
            elif x==0 and y != 0:
                bannos.append(y)
                habitaciones.append(round(float(y*ratio2500)))
            elif x != 0 and y == 0:
                bannos.append(round(x/ratio2500))
                habitaciones.append(x)
            else:
                bannos.append(round(bedsin2500/ratio2500))
                habitaciones.append(round(bedsin2500))
        elif 2500 >= z and z < 3000:
            if x != 0 and y != 0:
                bannos.append(y)
                habitaciones.append(x)
            elif x==0 and y != 0:
                bannos.append(y)
                habitaciones.append(round(float(y*ratio3000)))
            elif x != 0 and y == 0:
                bannos.append(round(x/ratio3000))
                habitaciones.append(x)
            else:
                bannos.append(round(bedsin3000/ratio3000))
                habitaciones.append(round(bedsin3000))
        else:
            if x != 0 and y != 0:
                bannos.append(y)
                habitaciones.append(x)
            elif x==0 and y != 0:
                bannos.append(y)
                habitaciones.append(round(float(y*ratio3500)))
            elif x != 0 and y == 0:
                bannos.append(round(x/ratio3500))
                habitaciones.append(x)
            else:
                bannos.append(round(bedsin3500/ratio3500))
                habitaciones.append(round(bedsin3500))
for t in d_ref_bed_bat_gross_type.iteritems():
    ref4.append(t[0])
    
    

dictcam = dict(zip(ref4, habitaciones))
dictban = dict(zip(ref4, bannos))

df1["bathh"] = df1.ref.map(dictban)
df1["habit"] = df1.ref.map(dictcam)


del df1["bedroom"]
del df1["bath"]
del df1["floor"]

#Create a feature that is the union between the latitude and longitude:
dictcoord= dict([(i,[a,b ]) for i, a,b in zip(df1.ref, df1.lat,df1.lng)])

coord = []
refcoor = []
for i, j in dictcoord.itervalues():
    x = str(i) + str(j)
    coord.append(x)
for p in dictcoord.iteritems():
    refcoor.append(p[0])
    
dictcoor = dict(zip(refcoor, coord))
df1["coord"] = df.ref.map(dictcoor)


#Function that allow us calculate the distance between 2 points(lat, lng):
def find_distance_approx(start_lat, start_lon, end_lat, end_lon):
    dist = vincenty((start_lat,start_lon), (end_lat, end_lon)).km
    return dist # in kilometers

dictcoordenadas = dict([(i,[a,b ]) for i, a,b in zip(df1.ref, df1.lat,df1.lng)])
dictmtr = dict([(i,[a,b ]) for i, a,b in zip(dfmtr.name, dfmtr.lat,dfmtr.lon)])

#Calculate the distance between each property and the closest MTR Station:
from tqdm import tqdm
dist = []
refdist = []
for i, j in tqdm(dictcoordenadas.itervalues()):
    dist_fin = []
    for u, w in dictmtr.itervalues():
        distance = find_distance_approx(i, j, u, w)
        dist_fin.append(distance)
    dist.append(min(dist_fin))
    
for p in dictcoordenadas.iteritems():
    refdist.append(p[0]) 
    
dictdistanc = dict(zip(refdist, dist))
df1["distance"] = df1.ref.map(dictdistanc)



dfinal = df1[df1.lng != 0]


#Change the coordinates features by one that each value is a label:
le = preprocessing.LabelEncoder()
le_train = le.fit_transform(dfinal.coord)
dfinal["trans"] = le_train


#More cleaning:
dfinal.type = dfinal.type.map({"Flat": 0, "House": 1})



dfinal["nivel"] = dfinal.floortype.map({"G/F": 0, "Low": 1, "Mid": 2, "High": 3})



edad = {10 : 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16:1, 17: 1, 18:1, 19:1, 20: 1}
agelista = []
dfinal.age_building = dfinal.age_building.fillna(0)

for i in dfinal.age_building:
    if i != 0:
        agelista.append(i)
    else:
        agelista.append( random.choice([x for x in edad for y in range(edad[x])]))
        
        
        
dfinal["edad"] = agelista


#Save all the work in a csv file called "trabfin":
dfinal.to_csv("datasets/trabfin")

#--------------------------------------------------------------------------------------------------------------